In [1]:
require 'rnn'

true	


In [776]:
function buildLSTM(vsize, edim, odim)
    local lstm = nn.Sequential()
    lstm:add(nn.LookupTableMaskZero(vsize, edim))
    lstm:add(nn.SplitTable(1, edim))
    lstm:add(nn.Sequencer(nn.LSTM(edim, edim)))
    lstm:add(nn.SelectTable(-1))
    return lstm
end

sentences = torch.LongTensor{{0, 1, 3, 4}, {2, 1, 4, 3}}:t()
summary = torch.LongTensor{{0, 0, 1, 4}, {0, 2, 3, 1}}:t()
query = torch.LongTensor{{0, 0, 4, 3}, {0, 1, 3, 2}}:t()
actions = torch.round(torch.rand(2, 1))
yrouge = torch.rand(2, 1)

batch_size = 2
vocab_size = 4
embed_dim = 10
outputSize = 1

lstm1 = buildLSTM(vocab_size, embed_dim, outputSize)
lstm2 = buildLSTM(vocab_size, embed_dim, outputSize)
lstm3 = buildLSTM(vocab_size, embed_dim, outputSize)

mlp1 = nn.Sequential()
mlp1:add(nn.Linear(1, embed_dim))

ParallelModel = nn.ParallelTable()
ParallelModel:add(lstm1)
ParallelModel:add(lstm2)
ParallelModel:add(lstm3)
ParallelModel:add(mlp1)

sep_preds = ParallelModel:forward({sentences, summary, query, actions})

print(sep_preds)

-- Collapsing the predictions and stacking the *columns* together
preds = nn.JoinTable(2):forward(sep_preds)
input_dim = preds:size()[2]
output_dim = 1
FinalMLP = nn.Sequential()
FinalMLP:add(nn.Linear(input_dim, output_dim))

finpreds = fullMLP:forward(preds)
print(finpreds)

--- Backprop the final layer
criterion = nn.MSECriterion()
err = criterion:forward(finpreds, yrouge)
grads = criterion:backward(finpreds, yrouge)
fullMLP:backward(preds, grads)
FinalMLP:updateParameters(0.05)
FinalMLP:zeroGradParameters()

-- Backprop the first layer ??? but how??

{
  1 : DoubleTensor - size: 2x10
  2 : DoubleTensor - size: 2x10
  3 : DoubleTensor - size: 2x10
  4 : DoubleTensor - size: 2x10
}
2.5052e+152
2.8376e+152
[torch.DoubleTensor of size 2x1]



# Think this finally works

In [793]:
function buildLSTM(vsize, edim, odim)
    local lstm = nn.Sequential()
    lstm:add(nn.LookupTableMaskZero(vsize, edim))
    lstm:add(nn.SplitTable(1, edim))
    lstm:add(nn.Sequencer(nn.LSTM(edim, edim)))
    lstm:add(nn.SelectTable(-1))
    return lstm
end

sentences = torch.LongTensor{{0, 1, 3, 4}, {2, 1, 4, 3}}:t()
summary = torch.LongTensor{{0, 0, 1, 4}, {0, 2, 3, 1}}:t()
query = torch.LongTensor{{0, 0, 4, 3}, {0, 1, 3, 2}}:t()
actions = torch.round(torch.rand(2, 1))
yrouge = torch.rand(2, 1)

batch_size = 2
vocab_size = 4
embed_dim = 10
outputSize = 1


In [794]:
lstm1 = buildLSTM(vocab_size, embed_dim, outputSize)
lstm2 = buildLSTM(vocab_size, embed_dim, outputSize)
lstm3 = buildLSTM(vocab_size, embed_dim, outputSize)

mlp1 = nn.Sequential()
mlp1:add(nn.Linear(1, embed_dim))

ParallelModel = nn.ParallelTable()
ParallelModel:add(lstm1)
ParallelModel:add(lstm2)
ParallelModel:add(lstm3)
ParallelModel:add(mlp1)

FinalMLP = nn.Sequential()
FinalMLP:add(ParallelModel)
FinalMLP:add(nn.JoinTable(2))
FinalMLP:add( nn.Linear(embed_dim * 4, 1) )

In [796]:
preds = FinalMLP:forward({sentences, summary, query, actions})

In [798]:
criterion = nn.MSECriterion()
err = criterion:forward(preds, yrouge)

In [804]:
grads = criterion:backward(preds, yrouge)

In [809]:
preds, grads

 0.1675
 0.2317
[torch.DoubleTensor of size 2x1]

-0.1694
 0.0662
[torch.DoubleTensor of size 2x1]



In [810]:
FinalMLP:backward({sentences, summary, query, actions}, grads)

...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 1 module of nn.Sequential:
In 1 module of nn.ParallelTable:
In 3 module of nn.Sequential:
...iscojavierarceo/torch/install/share/lua/5.1/rnn/LSTM.lua:184: assertion failed!
stack traceback:
	[C]: in function 'assert'
	...iscojavierarceo/torch/install/share/lua/5.1/rnn/LSTM.lua:184: in function '_updateGradInput'
	...eo/torch/install/share/lua/5.1/rnn/AbstractRecurrent.lua:59: in function 'updateGradInput'
	...avierarceo/torch/install/share/lua/5.1/rnn/Sequencer.lua:121: in function <...avierarceo/torch/install/share/lua/5.1/rnn/Sequencer.lua:106>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:55: in function <...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:50>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowError

In [797]:
FinalMLP:updateParameters(0.05)
FinalMLP:zeroGradParameters()

...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 1 module of nn.Sequential:
bad argument #2 to '?' (out of range at /Users/franciscojavierarceo/torch/pkg/torch/generic/Tensor.c:890)
stack traceback:
	[C]: at 0x01e2d7e0
	[C]: in function '__index'
	...erarceo/torch/install/share/lua/5.1/nn/ParallelTable.lua:19: in function 'updateGradInput'
	...scojavierarceo/torch/install/share/lua/5.1/nn/Module.lua:31: in function <...scojavierarceo/torch/install/share/lua/5.1/nn/Module.lua:29>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:88: in function 'backward'
	[string "criterion = nn.MSECriterion()..."]:4: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller

In [ ]:
sep_preds = ParallelModel:forward({sentences, summary, query, actions})

print(sep_preds)

-- Collapsing the predictions and stacking the *columns* together
preds = nn.JoinTable(2):forward(sep_preds)
input_dim = preds:size()[2]
output_dim = 1
FinalMLP = nn.Sequential()
FinalMLP:add(nn.Linear(input_dim, output_dim))

finpreds = fullMLP:forward(preds)
print(finpreds)

--- Backprop the final layer
criterion = nn.MSECriterion()
err = criterion:forward(finpreds, yrouge)
grads = criterion:backward(finpreds, yrouge)
FinalMLP:backward(preds, grads)
FinalMLP:updateParameters(0.05)
FinalMLP:zeroGradParameters()

-- Backprop the first layer ??? but how??

# MLP join table

In [238]:
x = torch.rand(9,1)
y = torch.rand(5,1)

hidden_size = 10
output_size = 5

mlp1 = nn.Sequential()
mlp1:add(nn.Linear(x:size()[2], output_size) )

mlp2 = nn.Sequential()
mlp2:add(nn.Linear(y:size()[2], output_size) )

p = nn.ParallelTable()
p:add(mlp1)
p:add(mlp2)
p:add(nn.JoinTable(2))

z = {x, y}
prds = p:forward(z)

-- mymodule = nn.JoinTable(2)
-- print(mymodule:forward(prds))

...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 3 module of nn.ParallelTable:
...javierarceo/torch/install/share/lua/5.1/nn/JoinTable.lua:24: attempt to get length of local 'input' (a nil value)
stack traceback:
	...javierarceo/torch/install/share/lua/5.1/nn/JoinTable.lua:24: in function <...javierarceo/torch/install/share/lua/5.1/nn/JoinTable.lua:21>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...erarceo/torch/install/share/lua/5.1/nn/ParallelTable.lua:12: in function 'forward'
	[string "x = torch.rand(9,1)..."]:19: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/instal

# MLP and LSTM

In [220]:
x = torch.LongTensor{{0, 1, 3, 4}, {2, 1, 4, 3}}:t()
z = torch.rand(2, 1)
y = torch.rand(2, 1)

batch_size = 2
vocab_size = 4
embed_dim = 10
outputSize = 1

lstm = nn.Sequential()
lstm:add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- returns a sequence-length x batch-size x embed_dim tensor
lstm:add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embed_dim entries
lstm:add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
lstm:add(nn.SelectTable(-1)) -- selects last state of the LSTM

mlp = nn.Sequential()
mlp:add(nn.Linear(1, embed_dim))

p = nn.ParallelTable()
p:add(lstm)
p:add(mlp)
-- p:add(nn.JoinTable(2))

ipreds = p:forward{x, z}
print(ipreds)

-- Collapsing the predictions
preds = nn.JoinTable(2):forward(p:forward{x, z})
input_dim = preds:size()[2]
output_dim = 1
fullMLP = nn.Sequential()
fullMLP:add(nn.Linear(input_dim, output_dim))

finpreds = fullMLP:forward(preds)

criterion = nn.MSECriterion()
err = criterion:forward(finpreds, y)
grads = criterion:backward(finpreds, y)
fullMLP:backward(preds, grads)
fullMLP:updateParameters(0.05)
fullMLP:zeroGradParameters()

{
  1 : DoubleTensor - size: 2x10
  2 : DoubleTensor - size: 2x10
}


In [223]:
ipreds

{
  1 : DoubleTensor - size: 2x10
  2 : DoubleTensor - size: 2x10
}


In [222]:
crits = nn.MSECriterion()
-- crits = nn.SequencerCriterion(crit2)
err = crits:forward(ipreds, {x, z} )

...ierarceo/torch/install/share/lua/5.1/nn/MSECriterion.lua:13: attempt to call field 'new' (a nil value)
stack traceback:
	...ierarceo/torch/install/share/lua/5.1/nn/MSECriterion.lua:13: in function 'forward'
	[string "crits = nn.MSECriterion()..."]:3: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0101c99b90: 

In [ ]:
grads2 = crit2:backward(preds, {x, z} )
p:backward(preds, grads2)
p:updateParameters(0.05)
p:zeroGradParameters()

# Let's try it a different way

In [486]:
function buildLSTM(vsize, edim, odim)
    local lstm = nn.Sequential()
    lstm:add(nn.LookupTableMaskZero(vsize, edim))
    lstm:add(nn.SplitTable(1, edim))
    lstm:add(nn.Sequencer(nn.LSTM(edim, edim)))
    lstm:add(nn.SelectTable(-1))
    lstm:add(nn.Linear(edim, odim))
    return lstm
end

In [487]:
batch_size = 2
vocab_size = 4
output_dim = 1
embed_dim = 6

In [488]:
lstm1 = buildLSTM(vocab_size, embed_dim, output_dim)
lstm2 = buildLSTM(vocab_size, embed_dim, output_dim)
lstm3 = buildLSTM(vocab_size, embed_dim, output_dim)

mlp = nn.Sequential()
mlp:add(nn.Linear(1, 6))

In [549]:
sentences = torch.LongTensor{{0, 1, 3, 4}, {2, 1, 4, 3}}
summary = torch.LongTensor{{0, 0, 1, 4}, {0, 2, 3, 1}}
query = torch.LongTensor{{0, 0, 0, 3}, {0, 1, 3, 2}}
actions = torch.round(torch.rand(2, 1))
ys = torch.rand(2, 1)

In [550]:
sentences, summary, query, actions, ys

 0  1  3  4
 2  1  4  3
[torch.LongTensor of size 2x4]

 0  0  1  4
 0  2  3  1
[torch.LongTensor of size 2x4]

 0  0  0  3
 0  1  3  2
[torch.LongTensor of size 2x4]

 0
 0
[torch.DoubleTensor of size 2x1]

 0.2916
 0.1514
[torch.DoubleTensor of size 2x1]



In [551]:
nn.SelectTable(-1):forward(nn.SplitTable(1, 6):forward(nn.LookupTableMaskZero(4, 6):forward(sentences:t())))

-0.7758 -0.2990 -0.1692 -1.7618  1.6158 -0.1298
 0.6905  1.1564 -0.2393 -0.8849  1.3835  0.4618
[torch.DoubleTensor of size 2x6]



In [552]:
lstm1:forward(sentences:t())

-0.2566
-0.4127
[torch.DoubleTensor of size 2x1]



In [647]:
ss = torch.cat(sentences, summary, 1)

In [658]:
ss

 0  1  3  4
 2  1  4  3
 0  0  1  4
 0  2  3  1
[torch.LongTensor of size 4x4]



In [657]:
query:resize(1, 4, 2)

(1,.,.) = 
  0  0
  0  3
  0  1
  3  2
[torch.LongTensor of size 1x4x2]



In [656]:
nn.SplitTable(1, 1):forward(ss)

{
  1 : LongTensor - size: 4
  2 : LongTensor - size: 4
  3 : LongTensor - size: 4
  4 : LongTensor - size: 4
}


In [557]:
vsize = 4
edim = 6

In [659]:
c = nn.ParallelTable() 
c:add(nn.Sequential())
c:add(nn.LookupTableMaskZero(vsize, edim))
c:add(nn.SplitTable(1, edim))
c:add(nn.Sequencer(nn.LSTM(edim, edim)))
c:add(nn.SelectTable(-1))

p = nn.ParallelTable()
p:add(nn.Sequential())
p:add(nn.LookupTableMaskZero(vsize, edim))
p:add(nn.SplitTable(1, edim))
p:add(nn.Sequencer(nn.LSTM(edim, edim)))
p:add(nn.SelectTable(-1))


mlp = nn.Sequential()
mlp:add(nn.SplitTable(1, 1))
-- mlp:add(c)
-- mlp:add(p)
-- mlp:add(nn.JoinTable(2))

In [660]:
pred = mlp:forward(ss)

In [661]:
ss, pred

 0  1  3  4
 2  1  4  3
 0  0  1  4
 0  2  3  1
[torch.LongTensor of size 4x4]

{
  1 : LongTensor - size: 4
  2 : LongTensor - size: 4
  3 : LongTensor - size: 4
  4 : LongTensor - size: 4
}


In [227]:
lstm1 = buildLSTM(vocab_size, embed_dim, output_dim)
lstm2 = buildLSTM(vocab_size, embed_dim, output_dim)
lstm3 = buildLSTM(vocab_size, embed_dim, output_dim)

In [228]:
p = nn.ParallelTable()
p:add(lstm1)
p:add(lstm2)

net = nn.Sequential()
net:add(p)
-- net:add(nn.JoinTable(2))
-- net:add(mlp)

In [ ]:
net:forward({sentence, summary})

In [ ]:
net:forward({sentence, summary, query})

In [240]:
c = nn.ParallelTable()      -- The two Tensor slices go through two different Linear
c:add(nn.Linear(10, 3))     -- Layers in Parallel
c:add(nn.Linear(10, 7))

p = nn.ParallelTable()      -- These tables go through two more linear layers separately
p:add(nn.Linear(3, 2))
p:add(nn.Linear(7, 1))

mlp = nn.Sequential()       -- Create a network that takes a Tensor as input
mlp:add(nn.SplitTable(2))
mlp:add(c)                  -- Outputing a table with 2 elements
mlp:add(p)
mlp:add(nn.JoinTable(1))    -- Finally, the tables are joined together and output.

pred = mlp:forward(torch.randn(10, 2))
print(pred)

x = torch.ones(10, 2)
y = torch.Tensor(3)
y:copy(x:select(2, 1):narrow(1, 1, 3))
pred = mlp:forward(x)

criterion = nn.MSECriterion()
local err = criterion:forward(pred, y)
local gradCriterion = criterion:backward(pred, y)
mlp:zeroGradParameters()
mlp:backward(x, gradCriterion)
mlp:updateParameters(0.05)

print(err)

 0.5559
 0.4333
 0.3092
[torch.DoubleTensor of size 3]



0.36783858107572	


In [333]:
nn.SplitTable(2):forward(torch.randn(10,2))

{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 10
}


In [ ]:
s